# Demo 1: Simple Customer Support Agent - Knowledge Sharing Session

## Use Case: E-commerce Customer Support Automation

**Business Problem:**
An e-commerce company receives hundreds of customer inquiries daily about:
- Order status and tracking
- Product information and FAQs
- Return/refund requests
- General support issues

**Solution:**
Build a simple AI agent that can handle common customer support tasks autonomously, reducing human workload and providing 24/7 support.

**Agent Capabilities:**
1. **FAQ Search** - Answer common questions from knowledge base
2. **Order Lookup** - Check order status and tracking information
3. **Ticket Creation** - Create support tickets for complex issues
4. **Product Search** - Find product information and availability

**Key Learning Goals:**
- Understand basic AI agent architecture without frameworks
- Learn function calling patterns with OpenAI
- See manual tool orchestration and error handling
- Foundation for comparing with framework implementations

## Import Dependencies

Import required libraries for basic AI agent implementation:
- **openai**: For GPT-4 API calls with function calling
- **json**: For parsing function arguments and results
- **typing**: For better code documentation
- **datetime**: For timestamp management
- **uuid**: For generating unique ticket IDs

In [1]:
import json
import openai
from typing import Dict, Any, List, Optional
from datetime import datetime
import uuid
from kaggle_secrets import UserSecretsClient

# Secure API key retrieval
user_secrets = UserSecretsClient()
openai_api_key = user_secrets.get_secret("openai_api_key")

print("Dependencies imported successfully")
print("OpenAI API key retrieved securely")
print("Ready to build simple customer support agent")

Dependencies imported successfully
OpenAI API key retrieved securely
Ready to build simple customer support agent


## Simple Customer Support Agent Class

**CustomerSupportAgent**: Basic implementation without frameworks
- Manual function calling orchestration
- Custom error handling and validation
- Simple data structures for customer data
- Direct OpenAI API integration

In [2]:
class SimpleCustomerSupportAgent:
    """
    Basic customer support agent using direct OpenAI function calling
    Demonstrates foundational patterns without frameworks
    """
    
    def __init__(self, openai_api_key: str):
        """
        Initialize the customer support agent
        
        Args:
            openai_api_key: OpenAI API key for GPT-4 access
        """
        self.client = openai.OpenAI(api_key=openai_api_key)
        
        # Mock customer database
        self.customer_db = {
            "john.doe@email.com": {
                "name": "John Doe",
                "orders": [
                    {"id": "ORD-001", "status": "shipped", "items": ["Laptop", "Mouse"], "total": 1200, "tracking": "TRK123456"},
                    {"id": "ORD-002", "status": "processing", "items": ["Headphones"], "total": 200, "tracking": None}
                ]
            },
            "jane.smith@email.com": {
                "name": "Jane Smith",
                "orders": [
                    {"id": "ORD-003", "status": "delivered", "items": ["Keyboard", "Monitor"], "total": 800, "tracking": "TRK789012"}
                ]
            }
        }
        
        # Mock FAQ database
        self.faq_db = {
            "shipping time": "Standard shipping takes 3-5 business days. Express shipping takes 1-2 business days.",
            "return policy": "You can return items within 30 days of delivery for a full refund. Items must be in original condition.",
            "warranty": "All products come with a 1-year manufacturer warranty. Extended warranties are available at checkout.",
            "payment methods": "We accept all major credit cards, PayPal, and Apple Pay. Payment is processed securely.",
            "customer service hours": "Our customer service team is available Monday-Friday 9AM-6PM EST. This AI agent provides 24/7 support."
        }
        
        # Mock product database
        self.product_db = {
            "laptop": {"name": "Gaming Laptop", "price": 1200, "stock": 15, "category": "Computers"},
            "mouse": {"name": "Wireless Mouse", "price": 25, "stock": 100, "category": "Accessories"},
            "headphones": {"name": "Noise-Canceling Headphones", "price": 200, "stock": 50, "category": "Audio"},
            "keyboard": {"name": "Mechanical Keyboard", "price": 150, "stock": 75, "category": "Accessories"},
            "monitor": {"name": "4K Monitor", "price": 650, "stock": 25, "category": "Displays"}
        }
        
        # Support tickets storage
        self.support_tickets = []
        
        print("Simple Customer Support Agent initialized")
        print(f"Customer database: {len(self.customer_db)} customers")
        print(f"FAQ database: {len(self.faq_db)} entries")
        print(f"Product database: {len(self.product_db)} products")

# Test initialization
test_agent = SimpleCustomerSupportAgent(openai_api_key)
print("Agent initialization complete")

Simple Customer Support Agent initialized
Customer database: 2 customers
FAQ database: 5 entries
Product database: 5 products
Agent initialization complete


## Tool Functions Implementation

**Core Support Functions:**
- **FAQ Search**: Search knowledge base for answers
- **Order Lookup**: Find customer order information
- **Ticket Creation**: Create support tickets for complex issues
- **Product Search**: Find product details and availability

In [3]:
def search_faq(self, query: str) -> Dict[str, Any]:
    """Search FAQ database for relevant answers"""
    print(f"Searching FAQ for: {query}")
    
    query_lower = query.lower()
    
    # Simple keyword matching
    for key, answer in self.faq_db.items():
        if any(word in query_lower for word in key.split()):
            return {
                "found": True,
                "question": key,
                "answer": answer,
                "source": "FAQ Database"
            }
    
    return {
        "found": False,
        "message": "No FAQ entry found for this query. Consider creating a support ticket for personalized assistance."
    }

def lookup_order(self, email: str, order_id: str = None) -> Dict[str, Any]:
    """Look up customer order information"""
    print(f"Looking up orders for: {email}")
    
    if email not in self.customer_db:
        return {
            "found": False,
            "message": "Customer not found. Please verify email address."
        }
    
    customer = self.customer_db[email]
    orders = customer["orders"]
    
    if order_id:
        # Look for specific order
        for order in orders:
            if order["id"] == order_id:
                return {
                    "found": True,
                    "customer_name": customer["name"],
                    "order": order
                }
        return {
            "found": False,
            "message": f"Order {order_id} not found for this customer."
        }
    else:
        # Return all orders
        return {
            "found": True,
            "customer_name": customer["name"],
            "orders": orders,
            "total_orders": len(orders)
        }

def create_support_ticket(self, email: str, subject: str, description: str, priority: str = "medium") -> Dict[str, Any]:
    """Create a new support ticket"""
    print(f"Creating support ticket for: {email}")
    
    ticket_id = f"TKT-{str(uuid.uuid4())[:8].upper()}"
    
    ticket = {
        "id": ticket_id,
        "email": email,
        "subject": subject,
        "description": description,
        "priority": priority,
        "status": "open",
        "created_at": datetime.now().isoformat(),
        "assigned_to": None
    }
    
    self.support_tickets.append(ticket)
    
    return {
        "success": True,
        "ticket_id": ticket_id,
        "message": f"Support ticket {ticket_id} created successfully. You will receive updates at {email}.",
        "estimated_response": "24 hours"
    }

def search_products(self, query: str) -> Dict[str, Any]:
    """Search for products in the catalog"""
    print(f"Searching products for: {query}")
    
    query_lower = query.lower()
    matching_products = []
    
    for product_key, product_info in self.product_db.items():
        if query_lower in product_key or query_lower in product_info["name"].lower() or query_lower in product_info["category"].lower():
            matching_products.append({
                "key": product_key,
                "name": product_info["name"],
                "price": product_info["price"],
                "stock": product_info["stock"],
                "category": product_info["category"],
                "available": product_info["stock"] > 0
            })
    
    if matching_products:
        return {
            "found": True,
            "products": matching_products,
            "count": len(matching_products)
        }
    else:
        return {
            "found": False,
            "message": "No products found matching your search. Try different keywords or browse our categories."
        }

# Add methods to SimpleCustomerSupportAgent class
SimpleCustomerSupportAgent.search_faq = search_faq
SimpleCustomerSupportAgent.lookup_order = lookup_order
SimpleCustomerSupportAgent.create_support_ticket = create_support_ticket
SimpleCustomerSupportAgent.search_products = search_products

print("Support tool functions implemented")
print("Available tools: FAQ search, order lookup, ticket creation, product search")

Support tool functions implemented
Available tools: FAQ search, order lookup, ticket creation, product search


## Function Call Schema Definition

**OpenAI Function Schemas:**
Define the tools that GPT-4 can use with detailed parameter specifications.
These schemas tell the AI when and how to use each customer support function.

In [4]:
# Define function schemas for OpenAI function calling
SimpleCustomerSupportAgent.tools = [
    {
        "type": "function",
        "function": {
            "name": "search_faq",
            "description": "Search the FAQ database for answers to common customer questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The customer's question or keywords to search for"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "lookup_order",
            "description": "Look up customer order information by email address and optionally order ID",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "Customer's email address"
                    },
                    "order_id": {
                        "type": "string",
                        "description": "Optional specific order ID to look up"
                    }
                },
                "required": ["email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_support_ticket",
            "description": "Create a support ticket for complex issues that need human assistance",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "Customer's email address"
                    },
                    "subject": {
                        "type": "string",
                        "description": "Brief subject line for the ticket"
                    },
                    "description": {
                        "type": "string",
                        "description": "Detailed description of the issue"
                    },
                    "priority": {
                        "type": "string",
                        "enum": ["low", "medium", "high", "urgent"],
                        "description": "Priority level of the ticket"
                    }
                },
                "required": ["email", "subject", "description"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_products",
            "description": "Search for products by name, category, or keywords",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Product name, category, or search keywords"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

print("Function calling schemas defined:")
for tool in SimpleCustomerSupportAgent.tools:
    print(f"  - {tool['function']['name']}: {tool['function']['description']}")
print("\nGPT-4 can now intelligently choose which tools to use based on customer inquiries")

Function calling schemas defined:
  - search_faq: Search the FAQ database for answers to common customer questions
  - lookup_order: Look up customer order information by email address and optionally order ID
  - create_support_ticket: Create a support ticket for complex issues that need human assistance
  - search_products: Search for products by name, category, or keywords

GPT-4 can now intelligently choose which tools to use based on customer inquiries


## Function Execution Router

**Tool Execution Handler:**
- Routes function calls to appropriate implementations
- Handles error cases and validation
- Provides centralized tool execution logic

In [5]:
def execute_function(self, function_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Execute customer support functions based on GPT-4 decisions"""
    
    print(f"Executing function: {function_name}")
    print(f"Arguments: {arguments}")
    
    try:
        if function_name == "search_faq":
            return self.search_faq(arguments["query"])
        
        elif function_name == "lookup_order":
            return self.lookup_order(
                arguments["email"],
                arguments.get("order_id")
            )
        
        elif function_name == "create_support_ticket":
            return self.create_support_ticket(
                arguments["email"],
                arguments["subject"],
                arguments["description"],
                arguments.get("priority", "medium")
            )
        
        elif function_name == "search_products":
            return self.search_products(arguments["query"])
        
        else:
            return {"error": f"Unknown function: {function_name}"}
            
    except Exception as e:
        print(f"Error executing {function_name}: {str(e)}")
        return {"error": f"Function execution failed: {str(e)}"}

# Add method to SimpleCustomerSupportAgent class
SimpleCustomerSupportAgent.execute_function = execute_function

print("Function execution router implemented")
print("Ready to handle GPT-4 function calls with error handling")

Function execution router implemented
Ready to handle GPT-4 function calls with error handling


## Main Chat Interface

**Customer Support Chat System:**
- Handles customer inquiries with natural language processing
- Automatically determines which tools to use
- Maintains professional customer service tone
- Provides comprehensive support responses

In [6]:
def chat(self, customer_message: str) -> str:
    """Main customer support chat interface"""
    
    print(f"Customer: {customer_message}")
    
    # Professional customer service system prompt
    system_prompt = """
    You are a professional customer support agent for an e-commerce company.
    
    Your capabilities:
    - search_faq: Answer common questions from the knowledge base
    - lookup_order: Check order status and details for customers
    - create_support_ticket: Create tickets for complex issues needing human help
    - search_products: Find product information and availability
    
    Guidelines:
    - Be helpful, professional, and empathetic
    - Always ask for email address when looking up orders
    - Create support tickets for complex issues you cannot resolve
    - Provide clear, accurate information
    - If you cannot help, explain why and offer alternatives
    - Be proactive in offering additional assistance
    """
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": customer_message}
    ]
    
    try:
        # Step 1: Get GPT-4 response with tools
        response = self.client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            tools=self.tools,
            tool_choice="auto"
        )
        
        assistant_message = response.choices[0].message
        
        # Step 2: Handle tool calls if any
        if assistant_message.tool_calls:
            print(f"GPT-4 wants to use {len(assistant_message.tool_calls)} tool(s)")
            
            # Add assistant's message to conversation
            messages.append({
                "role": "assistant",
                "content": assistant_message.content,
                "tool_calls": assistant_message.tool_calls
            })
            
            # Execute each tool call
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                function_result = self.execute_function(function_name, function_args)
                
                # Add function result to conversation
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(function_result)
                })
            
            # Step 3: Get final response with tool results
            final_response = self.client.chat.completions.create(
                model="gpt-4",
                messages=messages
            )
            
            final_content = final_response.choices[0].message.content
        else:
            print("Direct response without tools")
            final_content = assistant_message.content
        
        print(f"Agent: {final_content}")
        return final_content
        
    except Exception as e:
        error_message = f"I apologize, but I'm experiencing technical difficulties. Please try again or contact our support team directly. Error: {str(e)}"
        print(f"Error: {error_message}")
        return error_message

# Add method to SimpleCustomerSupportAgent class
SimpleCustomerSupportAgent.chat = chat

print("Customer support chat interface implemented")
print("Ready to handle customer inquiries with intelligent tool usage")

Customer support chat interface implemented
Ready to handle customer inquiries with intelligent tool usage


## Comprehensive Customer Support Demo

**Real-world Customer Scenarios:**
Test the agent with various customer support scenarios:
- Order inquiries and tracking
- Product questions and availability
- FAQ searches and general information
- Complex issues requiring ticket creation
- Multi-step customer interactions

In [7]:
def demo_customer_support_agent():
    """Demonstrate comprehensive customer support scenarios"""
    
    print("=== Simple Customer Support Agent Demo ===")
    print("Testing real-world customer service scenarios\n")
    
    # Initialize agent
    agent = SimpleCustomerSupportAgent(openai_api_key)
    
    # Realistic customer support scenarios
    customer_scenarios = [
        {
            "scenario": "Order Status Inquiry",
            "query": "Hi, I'd like to check the status of my recent order. My email is john.doe@email.com"
        },
        {
            "scenario": "Shipping Information Request",
            "query": "How long does standard shipping usually take?"
        },
        {
            "scenario": "Product Availability Check",
            "query": "Do you have any laptops in stock? I'm looking for a gaming laptop."
        },
        {
            "scenario": "Return Policy Question",
            "query": "What's your return policy? I might need to return an item."
        },
        {
            "scenario": "Complex Issue Requiring Human Help",
            "query": "I received a damaged laptop in my order ORD-001, and the screen has a crack. I need a replacement or refund. My email is john.doe@email.com"
        },
        {
            "scenario": "Specific Order Tracking",
            "query": "Can you help me track order ORD-003 for jane.smith@email.com?"
        }
    ]
    
    # Execute customer scenarios
    for i, scenario in enumerate(customer_scenarios, 1):
        print(f"\nScenario {i}/{len(customer_scenarios)}: {scenario['scenario']}")
        print("=" * 70)
        
        response = agent.chat(scenario['query'])
        
        print("=" * 70)
        print(f"Scenario {i} completed\n")
    
    # Show support tickets created
    if agent.support_tickets:
        print(f"\nSupport Tickets Created: {len(agent.support_tickets)}")
        for ticket in agent.support_tickets:
            print(f"  - {ticket['id']}: {ticket['subject']} ({ticket['priority']} priority)")
    
    print("\nCustomer support agent demo completed!")
    print("Demonstrated: FAQ search, order lookup, product search, ticket creation")

# Run the comprehensive demo
print("Starting comprehensive customer support demonstration...")
print("This will show how basic AI agents handle real customer service scenarios\n")

demo_customer_support_agent()

Starting comprehensive customer support demonstration...
This will show how basic AI agents handle real customer service scenarios

=== Simple Customer Support Agent Demo ===
Testing real-world customer service scenarios

Simple Customer Support Agent initialized
Customer database: 2 customers
FAQ database: 5 entries
Product database: 5 products

Scenario 1/6: Order Status Inquiry
Customer: Hi, I'd like to check the status of my recent order. My email is john.doe@email.com
GPT-4 wants to use 1 tool(s)
Executing function: lookup_order
Arguments: {'email': 'john.doe@email.com'}
Looking up orders for: john.doe@email.com
Agent: Hello John Doe, I've found your recent orders. Here's the status:

1. Order ID: ORD-001
   - Status: Shipped
   - Items: Laptop, Mouse
   - Total: $1200
   - Tracking Number: TRK123456

2. Order ID: ORD-002
   - Status: Processing
   - Items: Headphones
   - Total: $200
   - Tracking Number: Not available yet

The tracking number for your second order will be availa

## Key Takeaways and Architecture Analysis

### Key Takeaways from Simple Customer Support Agent:

**1. Basic Agent Architecture:**
   - **Manual Orchestration**: Direct OpenAI API calls with custom logic
   - **Function Calling**: GPT-4 decides which tools to use automatically
   - **Error Handling**: Custom error management and validation
   - **Data Management**: Simple in-memory data structures

**2. Implementation Characteristics:**
   - **Code Complexity**: More verbose, requires manual tool management
   - **Flexibility**: Full control over every aspect of behavior
   - **Maintenance**: More code to maintain and debug
   - **Scalability**: Requires additional work for complex workflows

**3. Real-world Business Value:**
   - **24/7 Availability**: Handles customer inquiries around the clock
   - **Cost Reduction**: Reduces human agent workload for routine tasks
   - **Consistency**: Provides consistent responses and service quality
   - **Scalability**: Can handle multiple customers simultaneously

**4. Limitations of Basic Implementation:**
   - **No Built-in Memory**: Each conversation is independent
   - **Manual Tool Integration**: Each tool requires custom implementation
   - **Limited Error Recovery**: Basic error handling without sophisticated retry logic
   - **No Advanced Workflows**: Single-turn interactions only

### Preparing for Framework Comparison:
This demo establishes the baseline for comparing with LangChain implementation in Demo 2, where we'll see:
- Reduced code complexity
- Built-in memory management
- Standardized tool integration
- Advanced workflow capabilities

### Business Impact Metrics:
- **Customer Satisfaction**: Faster response times (instant vs hours)
- **Operational Efficiency**: 60-80% reduction in routine inquiry handling
- **Cost Savings**: Lower customer service staffing requirements
- **Availability**: 24/7 support without human agents